In [1]:
from manim import *

Manim Community v0.8.0

In [2]:
from lib.imports import *
from lib.tools import *
from lib.utils import *
from lib.modules import *
from lib.data import *
from lib.xing import *
from lib.transform import *

In [3]:
G_list = load_G_list(data_path='data/rome', index_file='data_index.txt', cache='G_list', cache_prefix='cache/')

Load from 'cache/G_list.pickle'


In [4]:
data_list = generate_data_list(G_list, 
                               sparse=False, 
                               pivot_mode=None,
                               init_mode='pmds',
                               edge_index='full_edge_index',
                               edge_attr='full_edge_attr',
                               pmds_list=np.load('layouts/rome/pmds.npy', allow_pickle=True),
                               gviz_list=np.load('layouts/rome/gviz.npy', allow_pickle=True),
                               noisy_layout=True,
                               device='cpu', 
                               cache=True,
                               cache_prefix='cache/')

Load from 'cache/generate_data_list(list,sparse=False,pivot_mode=None,init_mode=pmds,edge_index=full_edge_index,edge_attr=full_edge_attr,pmds_list=ndarray,gviz_list=ndarray,noisy_layout=True,device=cpu).pickle'


In [5]:
normalize = Normalization(target_scale=3/np.sqrt(100), target_angle=np.pi/2)
shear = LinearTransformation(torch.tensor([[1/2, 0], 
                                           [1/2, 1]]))

In [14]:
idx = 422
init_scale = 1.5
start_x, end_x = -3.5, 3.5
colors = [WHITE, YELLOW, RED, GREEN, BLUE]
aggr_lag = 0.25
ending_wait = 2

In [17]:
%%manim Graph
class Graph(Scene):
    def construct(self):
        n_layers = len(colors)
        interval = (end_x-start_x)/(n_layers-1)
        edges = torch.tensor(list(G_list[idx].edges))
        raw_pos = normalize(data_list[idx].gt_pos, data_list[idx])
        pos = shear(raw_pos, data_list[idx])
        layer = Polygon([-1, -3, 0],
                        [1, -1, 0],
                        [1, 3, 0],
                        [-1, 1, 0], 
                        stroke_color=GREY, 
                        fill_color=GREY).set_opacity(0.5).shift(RIGHT*start_x)
        begin_lines = [Line([*p, 0], [*q, 0]).set_color(GRAY) for p, q in raw_pos[edges]*init_scale]    
        begin_dots = [Dot().shift([*p, 0]).set_color(colors[0]).scale(init_scale) for p in raw_pos*init_scale]
        lines = [Line([*p, 0], [*q, 0]).shift(RIGHT*start_x).set_color(GRAY) for p, q in pos[edges]]
        dots = [Dot().shift([*p, 0]).shift(RIGHT*start_x).set_color(colors[0]) for p in pos]
        end_lines = [Line([*p, 0], [*q, 0]).set_color(GRAY) for p, q in raw_pos[edges]*init_scale]    
        end_dots = [Dot().shift([*p, 0]).set_color(colors[-1]).scale(init_scale) for p in raw_pos*init_scale]    
        
        self.play(FadeIn(layer),
                  *[Transform(begin_line, line) for begin_line, line in zip(begin_lines, lines)],
                  *[Transform(begin_dot, dot) for begin_dot, dot in zip(begin_dots, dots)])
        
        groups = [[layer] + begin_lines + lines + begin_dots + dots]
        for x_pos, color in zip(np.linspace(start_x, end_x, n_layers)[:-1], colors[1:]):
            all_msg_dots = []
            animations = []
            for c_idx in range(len(pos)):
                msg_pos = pos[torch.tensor(list(dict(G_list[idx].adjacency())[c_idx].keys()))]
                msg_dots = [Dot().shift([*p, 0]).shift(RIGHT*x_pos).set_fill(color, opacity=0) for p in msg_pos]
                msg_shifts = [[pos[c_idx][0]-x, pos[c_idx][1]-y, 0] for x, y in msg_pos]
                all_msg_dots += msg_dots

    #             self.play(*[dot.animate.shift(shift).set_opacity(1) for dot, shift in zip(msg_dots, msg_shifts)])
                animations.append(AnimationGroup(
                    *[dot.animate.shift(shift).set_opacity(1) for dot, shift in zip(msg_dots, msg_shifts)]
                ))
            self.play(AnimationGroup(*animations, lag_ratio=aggr_lag))
            [d.set_opacity(0) for d in all_msg_dots]

            layer = Polygon([-1, -3, 0],
                            [1, -1, 0],
                            [1, 3, 0],
                            [-1, 1, 0],
                            stroke_color=GREY, 
                            fill_color=GREY).set_opacity(0.5).shift(RIGHT*(x_pos+interval))
            lines = [l.copy() for l in lines]
            dots = [d.copy().set_color(color) for d in dots]
            groups.append([layer] + lines + dots + all_msg_dots)
            
            self.play(FadeIn(layer),
                      *[l.animate.shift(RIGHT*interval) for l in lines],
                      *[d.animate.shift(RIGHT*interval) for d in dots]) 
            
        
        lines = [l.copy() for l in lines]
        dots = [d.copy().set_color(colors[-1]) for d in dots]
        self.add(*lines)
        self.add(*dots)
        
        self.play(*[FadeOut(obj) for group in groups for obj in group])
        self.play(*[Transform(line, end_line) for line, end_line in zip(lines, end_lines)],
                  *[Transform(dot, end_dot) for dot, end_dot in zip(dots, end_dots)])
        self.play(Wait().set_run_time(ending_wait))

[07/19/21 03:14:56] INFO     Animation 0 : Using cached data (hash :    ]8;id=1626678896.2735438-990071;file:///usr/local/anaconda3/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:100
                             450974505_3788851600_1777826343)

[07/19/21 03:15:00] INFO     Animation 1 : Partial movie file        ]8;id=1626678900.46289-291573;file:///usr/local/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:413
                             written in '/Users/chuhan/.lab/default/                         
                             deepgd/media/videos/deepgd/1080p60/part                         
                             ial_movie_files/Graph/3968072068_158466                         
                             1936_4041217613.mp4'

[07/19/21 03:15:01] INFO     Animation 2 : Partial movie file        ]8;id=1626678901.949933-254399;file:///usr/local/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:413
                             written in '/Users/chuhan/.lab/default/                         
                             deepgd/media/videos/deepgd/1080p60/part                         
                             ial_movie_files/Graph/3968072068_147478                         
                             9037_3253804927.mp4'

[07/19/21 03:15:06] INFO     Animation 3 : Partial movie file        ]8;id=1626678906.1844978-592764;file:///usr/local/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:413
                             written in '/Users/chuhan/.lab/default/                         
                             deepgd/media/videos/deepgd/1080p60/part                         
                             ial_movie_files/Graph/3968072068_363202                         
                             0190_760482752.mp4'

                    INFO     Animation 4 : Using cached data (hash :    ]8;id=1626678906.59024-980884;file:///usr/local/anaconda3/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:100
                             3968072068_4043822413_3472140813)

...

                    INFO     Animation 10 : Using cached data (hash :   ]8;id=1626678919.508485-990371;file:///usr/local/anaconda3/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:100
                             3968072068_1618126780_1450214847)

                    INFO     Animation 11 : Using cached data (hash :   ]8;id=1626678919.604373-964731;file:///usr/local/anaconda3/lib/python3.9/site-packages/manim/renderer/cairo_renderer.py\cairo_renderer.py]8;;\:100
                             3968072068_2057493503_1608529939)

                    INFO                                             ]8;id=1626678919.758744-801323;file:///usr/local/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:597
                             File ready at '/Users/chuhan/.lab/defau                         
                             lt/deepgd/media/videos/deepgd/1080p60/G                         
                             raph.mp4'

                    INFO     The partial movie directory is full (>  ]8;id=1626678919.7867541-425423;file:///usr/local/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:575
                             100 files). Therefore, manim has                                
                             removed 6 file(s) used by it the                                
                             longest ago.You can change this                                 
                             behaviour by changing max_files_cached                          
                             in config.

                    INFO     Rendered Graph                                      ]8;id=1626678919.789418-768833;file:///usr/local/anaconda3/lib/python3.9/site-packages/manim/scene/scene.py\scene.py]8;;\:228
                             Played 12 animations

In [61]:
%%manim TwoCircles
class TwoCircles(Scene):
    def construct(self):
        r = Circle(color=RED, fill_opacity=1, z_index=0).shift(RIGHT)
        g = Circle(color=GREEN, fill_opacity=1, z_index=1)
        self.add(r)
        self.add(g)
        self.play(r.animate.shift(LEFT))
        self.play(LaggedStart(r.animate.shift(LEFT)))

[07/17/21 12:46:36] INFO     Animation 0 : Partial movie file        ]8;id=1626540396.560949-334294;file:///usr/local/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:413
                             written in '/Users/chuhan/.lab/default/                         
                             deepgd/media/videos/deepgd/1080p60/part                         
                             ial_movie_files/TwoCircles/450974505_12                         
                             5445262_346011738.mp4'

[07/17/21 12:46:37] INFO     Animation 1 : Partial movie file        ]8;id=1626540398.002571-969114;file:///usr/local/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:413
                             written in '/Users/chuhan/.lab/default/                         
                             deepgd/media/videos/deepgd/1080p60/part                         
                             ial_movie_files/TwoCircles/3968072068_9                         
                             95073826_2444356619.mp4'

[07/17/21 12:46:38] INFO                                             ]8;id=1626540398.077981-352844;file:///usr/local/anaconda3/lib/python3.9/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:597
                             File ready at '/Users/chuhan/.lab/defau                         
                             lt/deepgd/media/videos/deepgd/1080p60/T                         
                             woCircles.mp4'

                    INFO     Rendered TwoCircles                                 ]8;id=1626540398.084202-633105;file:///usr/local/anaconda3/lib/python3.9/site-packages/manim/scene/scene.py\scene.py]8;;\:228
                             Played 2 animations